<a href="https://colab.research.google.com/github/Karan-Ghatt/ColourDetetion/blob/master/enrichment_file_comparison_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openpyxl

In [1]:
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
import pandas as pd
import numpy as np
from datetime import date

# Used in the interface file
import os
from pathlib import Path

START_ROW = 8
START_COL = 2

class SimilarPanda:
    """A class to check for differences between pandas Dataframes, and generate a report.

    Inputs:
        new: DataFrame
        old: DataFrame
        key_column: A reference to a common DataFrame column. If omitted, the
            data will be matched on the row position.

    Returns:
        Use output_excel to generate an Excel report.
    """

    def __init__(self,
                 new,
                 old,
                 key_column=None,
                 df_new_title='Not specified',
                 df_old_title='Not specified'
                 ):

        self.df_new_title = df_new_title
        self.df_old_title = df_old_title

        self.df_changes, (self.added_cols, self. deleted_cols) = (
            self.compare_dataframe_changes(new, old, key_column)
            )

    def compare_dataframe_changes(self, new, old, key_column=None):
        """ A function to compare two dataframes and return a summary of the changes
        Inputs:
        new: new dataframe
        old: old dataframe
        key_column: a column which is used to to match the data rows. If omitted, the rows will be matched by position.

        Returns:
        A dataframe with two columns to describe the changed cells across each row.
        A list of added columns
        A list of deleted columns
        """
        # new, old are dataframes
        # key_column is a dataframe column

        new = new.copy()
        old = old.copy()

        def get_row_status(record):
            """ A helper function to create a summary of whether a row has changed.
            Applied to each row in the dataframe"""
            explain = {'left_only': 'Row added', 'right_only': 'Row deleted', 'both': '-'}
            status = explain[record['_merge']]
            if status == '-':
                if len(record['Value changes']) > 0:
                    status = 'Row updated'
            return status

        def get_row_changes(record):
            """A helper function to create a dictionary describing the changes in each row.
            Applied to each row in the dataframe"""
            changes_dict = {}
            for col in cols_reduced:

                old_val = None
                if col in old_cols:
                    old_val = record[col + '_old']

                if pd.isna(old_val):
                    old_val = None
                if record[col] != old_val:
                    new_val = record[col]
                    if pd.isna(new_val):
                        new_val = None
                    if (old_val is not None) or (new_val is not None):
                        changes_dict[col] = {old_val: new_val}
            return changes_dict

        new_cols = list(new.columns)
        old_cols = list(old.columns)

        added_cols = [col for col in new_cols if col not in old_cols]
        deleted_cols = [col for col in old_cols if col not in new_cols]

        #  Exclude the key column, as this will always match
        cols_reduced = [col for col in new_cols if col != key_column]

        old['row_id'] = np.arange(old.shape[0])
        new['row_id'] = np.arange(new.shape[0])

        if key_column is None:
            key_column = 'row_id'

        df_differences = pd.merge(new, old, on=key_column, suffixes=('', '_old'), how='outer', indicator=True)
        df_differences.drop('row_id', axis=1)
        df_differences['Value changes'] = df_differences.apply(get_row_changes, axis=1)
        df_differences['Row status'] = df_differences.apply(get_row_status, axis=1)
        cols_to_return = new_cols
        cols_to_return.extend(deleted_cols)
        cols_to_return.extend(['Row status', 'Value changes'])

        return df_differences[cols_to_return], (added_cols, deleted_cols)

    def output_excel(self, output_file, worksheet_title='Data changes'):
        """Generates an output Excel file to explain the data changes
        Makes use of inbuilt styles: 'Good', 'Bad', and 'Neutral'. Might need to be changed in other languages"""

        def xlref(row, column, zero_indexed=True):
            if zero_indexed:
                row += 1
                column += 1
            return get_column_letter(column) + str(row)

        cols = self.df_changes.columns
        wb = Workbook()
        ws1 = wb.active
        ws1.title = worksheet_title

        for col_index, col in enumerate(cols):
            cell = ws1.cell(column=(col_index + START_COL), row=START_ROW)
            cell.value = col
            if col in self.added_cols:
                cell.style = 'Good'
            if col in self.deleted_cols:
                cell.style = 'Bad'

        for row_index, record in self.df_changes.iterrows():

            for col_index, col in enumerate(cols):
                val_to_print = record[col]
                if type(val_to_print) == dict:
                    val_to_print = str(val_to_print).replace('},', '},\n')
                cell = ws1.cell(column=(col_index + START_COL), row=(row_index + START_ROW + 1))
                cell.value = val_to_print
                if col in record['Value changes'].keys():
                    cell.style = 'Neutral'

                if  col =='Value changes' and len(record['Value changes']) > 0:
                    cell.style = 'Neutral'

                if col in self.added_cols:
                    cell.style = 'Good'
                if col in self.deleted_cols:
                    cell.style = 'Bad'

                if record['Row status'] == 'Row added':
                    cell.style = 'Good'
                if record['Row status'] == 'Row deleted':
                    cell.style = 'Bad'
                if cell.value == 'Row updated':
                    cell.style = 'Neutral'

        ws1.cell(column=2, row=2).value = 'Data comparison tool'
        for col_ref in range(2, 6):
            ws1.cell(column=col_ref, row=2).style = 'Headline 1'
        ws1.cell(column=2, row=3).value = 'New:'
        ws1.cell(column=3, row=3).value = self.df_new_title

        ws1.cell(column=2, row=4).value = 'Old:'
        ws1.cell(column=3, row=4).value = self.df_old_title

        ws1.cell(column=2, row=5).value = 'Generated:'
        ws1.cell(column=3, row=5).value = date.today().strftime("%Y-%m-%d")

        ws1.cell(column=2, row=6).value = 'DEV COPY'

        table_style = TableStyleInfo(
            name='TableStyleMedium9',
            showFirstColumn=False,
            showLastColumn=False,
            showRowStripes=True,
            showColumnStripes=False)

        top_left_cell = xlref(START_ROW, START_COL, zero_indexed=False)
        top_right_cell = xlref(START_ROW + len(self.df_changes), START_COL + len(cols) - 1, zero_indexed=False)
        table_ref = top_left_cell + ':' + top_right_cell
        tab = Table(displayName='TableDifferences', ref=table_ref)
        tab.tableStyleInfo = table_style
        ws1.add_table(tab)

        wb.save(filename=output_file)


def get_all_tables_in_excel(filename):
    """ A helper function to extract all Named DataTables from an Excel file,
    and store these in a dictionary (key = Table name, value = Dataframe)"""

    wb = load_workbook(filename, data_only=True)
    mapping = {}

    for ws in wb.worksheets:

        for entry, data_boundary in ws.tables.items():
            # Parse the data within the ref boundary
            data = ws[data_boundary]
            # The inner list comprehension gets the values for each cell in the table
            content = [[cell.value for cell in ent] for ent in data]

            header = content[0]
            rest = content[1:]

            df = pd.DataFrame(rest, columns=header)
            mapping[entry] = df

    return mapping


if __name__ == '__main__':
    print('Running the file directly gives no data')

In [10]:
# Load the Excel files
input_file_path_old = '/content/test_linakge_one.xlsx'
input_file_path_new = '/content/test_linakge_two.xlsx'

all_dfs_old = get_all_tables_in_excel(input_file_path_old)
all_dfs_new = get_all_tables_in_excel(input_file_path_new)

# Access the 'Test1_Linkage' sheets
df_old = all_dfs_old['Test1_Linkage']
df_new = all_dfs_new['Test1_Linkage']

print('Old - full')
print(df_old.head())
print('...')
print('New - full')
print(df_new.head())

# For old dataframe: filter out rows where 'Digitized?' is NaN or blank and 'Duplicate' is 'N'
# Drop rows where 'Digitized?' is NaN

#df_old = df_old.dropna(subset=['Digitized?'])
# Remove rows where 'Digitized?' is blank
#df_old = df_old.loc[df_old['Digitized?'].str.strip() != ""]
#df_old = df_old.loc[df_old['Duplicate'] == 'N']

# For new dataframe: filter out rows where 'Digitized?' is NaN or blank and 'Duplicate' is 'N'
# Drop rows where 'Digitized?' is NaN
#df_new = df_new.dropna(subset=['Digitized?'])
 # Remove rows where 'Digitized?' is blank
#df_new = df_new.loc[df_new['Digitized?'].str.strip() != ""]
#df_new = df_new.loc[df_new['Duplicate'] == 'N']

# Display first 3 rows of each dataframe
#print('Old - filtered')
#print(df_old.head(3))
#print('...')
#print('New - filtered')
#print(df_new.head(3))


/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Old - full
              Wave                                          Sub-Group  \
0  W6 - Futurelife  Other - Futurelife Durban - Maize Meal ; Other...   
1  W6 - Futurelife  Cereals - RTE ; Other - Futurelife Durban - Ma...   
2  W6 - Futurelife  Other - Futurelife Durban - Maize Meal ; Other...   
3  W6 - Futurelife  Other - Futurelife Durban - Maize Meal ; Other...   
4  W6 - Futurelife  Other - Futurelife Durban - Maize Meal ; Other...   

    Spec Type          SKU  Ingredient ERP Code  \
0  Ingredient        61152               426359   
1  Ingredient  61152;54939               426297   
2  Ingredient         None               426288   
3  Ingredient        54939               426291   
4  Ingredient        54939               426293   

                       Ingredient ERP Name  \
0  FLAVOUR:SWEETNESS MODIFIER:IG-201-713-7   
1                          FRUTAFIT:INULIN   
2                        SUGAR:ICING:BROWN   
3             SALT:NAMIBIAN FINE:C701:50KG   
4            

In [11]:
df_old.head()

,Wave,Sub-Group,Spec Type,SKU,Ingredient ERP Code,Ingredient ERP Name,R&D Ingredient Code,Alternative,SAP CODE,Primary or Alternative Ingredient?,...,Lacto-ovo Vegetarianism,"Supplier Name / Supplier Site Name\n(List both BROKER and Supplier Site Name, if applicable)",Supplier Item Description,Supplier Item Code,Material Qualification Status,Additional Comments,Complete SME?,Digitized?,Unique ID,Duplicate
0,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,61152,426359,FLAVOUR:SWEETNESS MODIFIER:IG-201-713-7,426359 - 31141 - FL01,None,NaN,None,...,Yes,Givaudan South Africa (Pty) Ltd\nGIVAUDAN Sch...,Givaudan Cereal Modifier Flavour RF-935,IG-201-713-7,,None,No,None,1,N
1,W6 - Futurelife,Cereals - RTE ; Other - Futurelife Durban - Ma...,Ingredient,61152;54939,426297,FRUTAFIT:INULIN,426297 - Sensus - FL01 | FL01 - 11030690,None,1.100000e+11,None,...,No Entry,Broker: Brenntag South Africa\nSupplier site: ...,Frutafit(R) HD native inulin.,Frutafit® HD\nversion 2018.01/gbl (January 2018),None,None,No,None,2,N
2,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,None,426288,SUGAR:ICING:BROWN,426288 - MP FOOD - FL01 | FL01 - 11030690,None,NaN,None,...,No Entry,Broker: Bragan Ingredients (Pty) Ltd\nSupplier...,None,None,None,None,No,None,3,N
3,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,54939,426291,SALT:NAMIBIAN FINE:C701:50KG,426291 - 91785 - FL01 | FL01 - 11030690,None,NaN,None,...,No Entry,Manufacturer: Sealand Salt Marketing Pty Ltd\n...,Namibian Fine N/I,Not specified,None,None,No,None,4,N
4,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,54939,426293,STABILISER,426293 - Anqiu Eagle Cellulose - FL01 | FL01 -...,Alternative 1,NaN,Primary,...,No Entry,Suppllier Site: SinoCMC Holdings Limited\nRM 4...,White or off white Powder,F1-4000,None,None,No,None,5,N


In [12]:
df_new.head()

,Wave,Sub-Group,Spec Type,SKU,Ingredient ERP Code,Ingredient ERP Name,R&D Ingredient Code,Alternative,SAP CODE,Primary or Alternative Ingredient?,...,Lacto-ovo Vegetarianism,"Supplier Name / Supplier Site Name\n(List both BROKER and Supplier Site Name, if applicable)",Supplier Item Description,Supplier Item Code,Material Qualification Status,Additional Comments,Complete SME?,Digitized?,Unique ID,Duplicate
0,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,61152,426359,FLAVOUR:SWEETNESS MODIFIER:IG-201-713-7,426359 - 31141 - FL01,None,NaN,None,...,Yes,Givaudan South Africa (Pty) Ltd\nGIVAUDAN Sch...,Givaudan Cereal Modifier Flavour RF-935,IG-201-713-7,,None,No,None,1,N
1,W6 - Futurelife,Cereals - RTE ; Other - Futurelife Durban - Ma...,Ingredient,61152;54939,426297,FRUTAFIT:INULIN,426297 - Sensus - FL01 | FL01 - 11030690,None,1.100000e+11,None,...,No Entry,Broker: Brenntag South Africa\nSupplier site: ...,Frutafit(R) HD native inulin.,Frutafit® HD\nversion 2018.01/gbl (January 2018),None,None,No,None,2,N
2,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,None,426288,SUGAR:ICING:BROWN,426288 - MP FOOD - FL01 | FL01 - 11030690,None,NaN,None,...,No Entry,Broker: Bragan Ingredients (Pty) Ltd\nSupplier...,None,None,None,None,No,None,3,N
3,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,54939,426291,SALT:NAMIBIAN FINE:C701:50KG,426291 - 91785 - FL01 | FL01 - 11030690,None,NaN,None,...,No Entry,Manufacturer: Sealand Salt Marketing Pty Ltd\n...,Namibian Fine N/I,Not specified,None,None,No,None,4,Y
4,W6 - Futurelife,Other - Futurelife Durban - Maize Meal ; Other...,Ingredient,54939,426293,STABILISER,426293 - Anqiu Eagle Cellulose - FL01 | FL01 -...,Alternative 1,NaN,Primary,...,No Entry,Suppllier Site: SinoCMC Holdings Limited\nRM 4...,White or off white Powder,F1-4000,None,None,No,None,5,N


In [14]:
import datetime
ct = datetime.datetime.now()

output_file_path = f'example_output_differences_{ct}.xlsx'
key_column = 'Unique ID'
df_old_title = input_file_path_old[9:]
df_new_title = input_file_path_new[9:]

df_new.replace({np.nan: None}, inplace=True)
df_old.replace({np.nan: None}, inplace=True)

sp = SimilarPanda(new = df_new,
                             old = df_old,
                             key_column = key_column,
                             df_new_title = df_new_title,
                             df_old_title = df_old_title)
sp.output_excel(output_file_path)

#os.startfile(output_file_path)

print("done")


done


In [19]:
import openpyxl


openpyxl.__version__


'1.26.4'

In [20]:
pd.__version__



'2.1.4'

In [21]:
np.__version__

'1.26.4'